In [1]:
%pip install sentence_transformers
%pip install spacy
%pip install matplotlib
%pip install seaborn 
%pip install matplotlib
%pip install scipy
%pip install langchain
import spacy
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
# Library to import pre-trained model for sentence embeddings
from sentence_transformers import SentenceTransformer
# Calculate similarities between sentences
from sklearn.metrics.pairwise import cosine_similarity
# Visualization library
import seaborn as sns
import matplotlib.pyplot as plt
# package for finding local minimas
from scipy.signal import argrelextrema
import math

# Download the English language model 'en_core_web_sm'
spacy.cli.download("en_core_web_sm")

# Load the downloaded model
nlp = spacy.load("en_core_web_sm")


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.9 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
#load data
doc_embeddings = catalog.load("doc_embeddings")
extract_data = catalog.load("extract_data")

[12/12/23 15:30:11] INFO     Loading data from 'doc_embeddings' (CSVDataset)...                 ]8;id=999873;file:///Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=893096;file:///Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[12/12/23 15:30:53] INFO     Loading data from 'extract_data' (CSVDataset)...                   ]8;id=727399;file:///Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=773788;file:///Users/Kenneth/opt/anaconda3/envs/pubMedNLP/lib/python3.11/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [12]:
# Count NAs in the 'Abstract' column
count_na = extract_data['Abstract'].isna().sum()
print(f"percentage of missing abstracts: {100*(count_na/len(extract_data['Abstract']))}%")

percentage of missing abstracts: 6.360803252636575%


In [19]:
# Count incomplete Abstracts 
count_dots = extract_data['Abstract'].dropna().str.contains('\.\.\.').sum()
print(f"percentage of incomplete abstracts: {100*(count_dots.sum()/len(extract_data['Abstract'].dropna()))}%")

percentage of incomplete abstracts: 0.10524343743801105%


## Obtain paragraphs of raw abstracts

In [20]:
model = SentenceTransformer("pritamdeka/S-PubMedBert-MS-MARCO")

In [21]:
def process(text: str):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(str(text))
    sents = [str(sent) for sent in doc.sents]
    embeddings = model.encode(sents)
    return list(doc.sents), embeddings

In [33]:
def cluster_text(sents, vecs, threshold):
    clusters = [[0]]
    for i in range(1, len(sents)):
        if cosine_similarity(vecs[i].reshape(1, -1), vecs[i-1].reshape(1, -1)) < threshold:
            clusters.append([])
        clusters[-1].append(i)
    return clusters

In [34]:

def get_paragraphs(abstract_text):

    # Initialize the clusters lengths list and final texts list
    clusters_lens = []
    final_texts = []

    # Process the chunk
    initial_threshold = 0.8
    max_iteration = 5
    sents, vecs = process(abstract_text)

    # Cluster the sentences
    clusters = cluster_text(sents, vecs, initial_threshold)

    for cluster in clusters:
        cluster_txt = ' '.join([sents[i].text for i in cluster])
        cluster_len = len(cluster_txt)
        print(f"cluster_txt: {cluster_txt}")


        # Check if the cluster is too long
        if cluster_len > 700:
            iterator = 1
            # Track the best subcluster lengths
            best_num_paragraphs = 500 #initialize with a large value
            best_div_texts = [] 
            while cluster_len > 700:
                div_lens = []
                div_texts = []
                len_collector = []
                threshold = min(initial_threshold+(0.02*iterator), 0.95)
                sents_div, vecs_div = process(cluster_txt)
                reclusters = cluster_text(sents_div, vecs_div, threshold)

                for subcluster in reclusters:
                    div_txt = ' '.join([sents_div[i].text for i in subcluster])
                    div_len = len(div_txt)
                    len_collector.append(div_len)
                    print(f"div_len: {div_len}") #for debugging

                    if div_len > 60 and div_len < 800:
                        div_lens.append(div_len)
                        div_texts.append(div_txt)

                    cluster_len = max(len_collector)

                iterator+=1 #for debugging
                print(f"subclusters {iterator}: {div_lens}")
            
            clusters_lens.extend(div_lens) 
            final_texts.extend(div_texts)  
        else:
            clusters_lens.append(cluster_len)
            final_texts.append(cluster_txt)   
    return final_texts

In [37]:
test_df = extract_data.dropna().head(3)

In [40]:
test_df['paragraphs'] = test_df['Abstract'].apply(get_paragraphs)

cluster_txt: Most empirical work in human categorization has studied learning in either fully supervised or fully unsupervised scenarios. Most real-world learning scenarios, however, are semi-supervised: Learners receive a great deal of unlabeled information from the world, coupled with occasional experiences in which items are directly labeled by a knowledgeable source. A large body of work in machine learning has investigated how learning can exploit both labeled and unlabeled data provided to a learner. Using equivalences between models found in human categorization and machine learning research, we explain how these semi-supervised techniques can be applied to human learning. A series of experiments are described which show that semi-supervised learning models prove useful for explaining human behavior when exposed to both labeled and unlabeled data. We then discuss some machine learning models that do not have familiar human categorization counterparts. Finally, we discuss some ch

In [45]:
test_df['paragraphs'][98]


[
    'Online gossip has been recognized as small talk on social networking sites (SNSs) that influences consumer behavior, but little attention has been paid to its role. This study makes three theoretical predictions: (a) propensity to gossip online leads to greater information value, entertainment value, and friendship value; (b) upon exposure to a high-involvement product, online gossipers are more willing to spread such information through electronic word-of-mouth (eWOM) in search of prestige or fame as a knowledge expert; and (c) this tendency will be more pronounced when they are connected with strong ties (rather than weak ties) and belong to a large network (rather than a small network).',
    'An experimental survey was conducted with a scenario method. In total, 818 general consumers participated in the survey.',
    'A multivariate analysis of variance (ANOVA) provides empirical support for prediction (1). With regard to predictions (2) and (3), a series of three-way and t